<a href="https://colab.research.google.com/github/luckyyyman/git-workshop/blob/main/RealEstate_EDAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Real Estate Exploratory Data Analysis and Visualization (EDAV)

**Project ID:** CBIT/IT-1/EDAV/2025/CEP-10  
**Roll No:** 160124737058  
**Name:** Sujal Prakash Vuba  
**Department:** Information Technology  
**Institution:** Chaitanya Bharathi Institute of Technology (A), Hyderabad  

## Problem Overview

The objective of this project is to analyze a real estate dataset and extract meaningful insights about property prices, locations, and types. We will perform the following tasks:

1. Calculate the average property price using NumPy (Bloom’s Level: 3)  
2. Filter properties by city and property type using Pandas (Bloom’s Level: 3)  
3. Handle missing property attribute data (Bloom’s Level: 4)  
4. Group properties by city and locality (Bloom’s Level: 4)  
5. Plot price trends for top cities using Matplotlib (Bloom’s Level: 5)  

## Dataset

A dataset of 500 real estate properties is used for analysis.  
**Download link:** [Real Estate Dataset](https://drive.google.com/file/d/1JIfla9Dt68jSlHKjvZbyyxjeovyOOcVC/view?usp=s)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

uploaded = files.upload()
file_name = "Hyderabad_Real_Estate_Big_Dataset.xlsx"
df = pd.read_excel(file_name)

df.columns = [c.strip() for c in df.columns]

def find_col(names):
    for n in names:
        for c in df.columns:
            if n.lower() in c.lower():
                return c
    return None

price_col = find_col(['price', 'amount', 'sale'])
city_col = find_col(['city', 'location', 'area', 'place'])
type_col = find_col(['type', 'property'])
locality_col = find_col(['locality', 'neighbourhood'])

df['Price'] = (
    df[price_col].astype(str)
    .str.replace(r'[^\d.]', '', regex=True)
    .replace('', np.nan)
    .astype(float)
)
df['City'] = df[city_col].astype(str).fillna('Unknown')
df['Type'] = df[type_col].astype(str).fillna('Unknown')
df['Locality'] = df[locality_col].astype(str).fillna('Unknown')

print("Detected columns:")
print("Price:", price_col, "| City:", city_col, "| Type:", type_col, "| Locality:", locality_col)

# Q1: Calculate average property price with NumPy
mean_price = np.mean(df['Price'])
median_price = np.median(df['Price'])
std_price = np.std(df['Price'])
print("\nQ1 → Average Price (Mean):", round(mean_price, 2))
print("Median:", round(median_price, 2), "| Std Dev:", round(std_price, 2))

# Q2: Filter properties by city and type using Pandas
top_city = df['City'].mode()[0]
top_type = df['Type'].mode()[0]
filtered = df[(df['City'] == top_city) & (df['Type'] == top_type)]
print(f"\nQ2 → Properties in {top_city} of type {top_type}: {len(filtered)} found.")
display(filtered.head(5))

# Q3: Handle missing property attribute data
print("\nMissing values before:")
print(df.isnull().sum())

df['Price'].fillna(df['Price'].median(), inplace=True)
df['Locality'].fillna('Unknown', inplace=True)
df['Type'].fillna('Unknown', inplace=True)

print("\nMissing values after:")
print(df.isnull().sum())

# Q4: Group properties by city and locality
grouped = df.groupby(['City', 'Locality']).agg(
    Listings=('Price', 'count'),
    Avg_Price=('Price', 'mean'),
    Median_Price=('Price', 'median')
).reset_index()

print("\nQ4 → Top localities by listings:")
display(grouped.sort_values('Listings', ascending=False).head(10))

# Q5: Plot price trends for top cities using Matplotlib
plt.figure(figsize=(10, 5))
avg_city_price = df.groupby('City')['Price'].mean().sort_values(ascending=False)
avg_city_price.plot(kind='bar')
plt.title("Average Property Price by City")
plt.xlabel("City")
plt.ylabel("Average Price (INR)")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 5))
top_cities = df['City'].value_counts().head(5).index
data = [df[df['City'] == c]['Price'] for c in top_cities]
plt.boxplot(data, labels=top_cities, showfliers=False)
plt.title("Price Distribution of Top 5 Cities")
plt.ylabel("Price (INR)")
plt.show()

print("✅ All five questions executed successfully!")


In [ ]:
### Question 1: Calculate average property price with NumPy (BL-3)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("real_estate_data.csv")

average_price = np.mean(df['Price'])
print("Average Property Price:", average_price)

median_price = np.median(df['Price'])
print("Median Property Price:", median_price)

price_std = np.std(df['Price'])
print("Price Standard Deviation:", price_std)

sns.histplot(df['Price'], bins=40, kde=True, color='skyblue')
plt.title("Distribution of Property Prices")
plt.show()


In [ ]:
### Question 2: Filter properties by city and type in Pandas (BL-3)

filtered_props = df[(df['City'] == 'Hyderabad') & (df['Property_Type'] == 'Apartment')]
filtered_props.head(10)

city_counts = df['City'].value_counts()
plt.figure(figsize=(10,5))
sns.barplot(x=city_counts.index, y=city_counts.values, palette="viridis")
plt.title("Number of Properties per City")
plt.show()

type_counts = df['Property_Type'].value_counts()
plt.figure(figsize=(10,5))
sns.barplot(x=type_counts.index, y=type_counts.values, palette="magma")
plt.title("Number of Properties per Property Type")
plt.xticks(rotation=45)
plt.show()


In [ ]:
### Question 3: Handle missing property attribute data (BL-4)


numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

missing_values = df.isnull().sum()
missing_values

sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='coolwarm')
plt.title("Missing Values After Handling")
plt.show()


In [ ]:
### Question 4: Group properties by city and locality (BL-4)

grouped = df.groupby(['City', 'Locality'])['Price'].mean().reset_index()
grouped_top = grouped.sort_values(by='Price', ascending=False).head(15)
grouped_top

plt.figure(figsize=(12,6))
sns.barplot(x='Locality', y='Price', hue='City', data=grouped_top)
plt.title("Top Localities by Average Property Price")
plt.xticks(rotation=45)
plt.show()

grouped_count = df.groupby(['City', 'Locality']).size().reset_index(name='Count')
grouped_count_top = grouped_count.sort_values(by='Count', ascending=False).head(15)
plt.figure(figsize=(12,6))
sns.barplot(x='Locality', y='Count', hue='City', data=grouped_count_top)
plt.title("Top Localities by Number of Properties")
plt.xticks(rotation=45)
plt.show()


In [ ]:
### Question 5: Plot price trends for top cities using Matplotlib (BL-5)

top_cities = df.groupby('City')['Price'].mean().sort_values(ascending=False).head(5)
plt.figure(figsize=(10,6))
plt.bar(top_cities.index, top_cities.values, color='skyblue')
plt.title("Average Property Prices in Top Cities")
plt.xlabel("City")
plt.ylabel("Average Price")
plt.show()

plt.figure(figsize=(10,6))
for city in top_cities.index:
    city_data = df[df['City'] == city]
    city_data_sorted = city_data.sort_values(by='Date', ascending=True) if 'Date' in df.columns else city_data
    plt.plot(range(len(city_data_sorted)), city_data_sorted['Price'], label=city)
plt.title("Price Trends for Top Cities")
plt.xlabel("Property Index")
plt.ylabel("Price")
plt.legend()
plt.show()
